### DB연결

In [71]:
import pandas as pd
from sqlalchemy import create_engine, text

# 엑셀 파일 로딩
file_path = "processed_output.xlsx"
df = pd.read_excel(file_path)

# 문자열 컬럼 강제 변환 (한글 오류 방지)
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].astype(str)

# MariaDB 접속 정보
user = "root"
password = "1234"
host = "localhost"
port = 3306
database = "nav7"

# SQLAlchemy 연결
conn_str = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(conn_str)

def execute_sql(sql_query: str):
    with engine.connect() as conn:
        # text()로 감싸야 안전
        result = conn.execute(text(sql_query))
        # SELECT 쿼리면 결과를 fetch
        if sql_query.strip().lower().startswith("select"):
            rows = result.fetchall()
            # 컬럼명 가져오기
            columns = result.keys()
            # 결과 출력 또는 반환
            return columns, rows
        else:
            # SELECT 아닌 쿼리는 영향받은 row 수 반환
            return result.rowcount


# 테이블 이름
table_name = "project"


# 테이블 생성 및 데이터 삽입
df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)
df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print(f"✅ MariaDB에 '{table_name}' 테이블 생성 및 데이터 입력 완료.")


✅ MariaDB에 'project' 테이블 생성 및 데이터 입력 완료.


In [5]:
import pandas as pd
from sqlalchemy import create_engine, text

# # 엑셀 파일 로딩
# file_path = "processed_output.xlsx"
# df = pd.read_excel(file_path)

# # ID 컬럼이 NaN인 행 삭제
# df = df.dropna(subset=['ID'])
# print(f"📊 NaN 값 제거 후 총 {len(df)}개 행")

# # profileId 컬럼 추가 - 같은 ID값에 대해 동일한 profileId 부여
# unique_ids = df['ID'].unique()
# id_to_profile = {id_val: idx + 1 for idx, id_val in enumerate(unique_ids)}
# df['profileId'] = df['ID'].map(id_to_profile)

# # profileId를 첫 번째 컬럼으로 이동
# cols = ['profileId'] + [col for col in df.columns if col != 'profileId']
# df = df[cols]

# # 문자열 컬럼 강제 변환 (한글 오류 방지)
# for col in df.columns:
#     if df[col].dtype == object:
#         df[col] = df[col].astype(str)

# MariaDB 접속 정보
user = "root"
password = "1234"
host = "localhost"
port = 3306
database = "nav7"

# SQLAlchemy 연결
conn_str = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(conn_str)

def execute_sql(sql_query: str):
    with engine.connect() as conn:
        # text()로 감싸야 안전
        result = conn.execute(text(sql_query))
        # SELECT 쿼리면 결과를 fetch
        if sql_query.strip().lower().startswith("select"):
            rows = result.fetchall()
            # 컬럼명 가져오기
            columns = result.keys()
            # 결과 출력 또는 반환
            return columns, rows
        else:
            # SELECT 아닌 쿼리는 영향받은 row 수 반환
            return result.rowcount

# # 테이블 이름
# table_name = "project"

# # 테이블 생성 및 데이터 삽입
# df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)
# df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# print(f"✅ MariaDB에 '{table_name}' 테이블 생성 및 데이터 입력 완료.")
# print(f"📊 총 {len(unique_ids)}개의 고유 ID에 대해 profileId 할당 완료")

# # profileId 할당 결과 확인
# print("\n📋 profileId 할당 결과:")
# for id_val, profile_id in sorted(id_to_profile.items()):
#     count = (df['ID'] == id_val).sum()
#     print(f"ID: {id_val} -> profileId: {profile_id} (총 {count}개 행)")

### OpenAI Key


In [ ]:
import openai
client = openai.OpenAI(api_key="")


### 개인정보 적용 (쿼리 재생성)

In [7]:
from sqlalchemy import text

def fetch_project_by_id_sql(project_id: str):
    sql = f"SELECT * FROM project WHERE ID = '{project_id}'"
    try:
        result = execute_sql(sql)
        if isinstance(result, tuple):
            columns, rows = result
            if rows:
                return dict(zip(columns, rows[0]))  # 첫 번째 row만 dict로 변환
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"❌ Error fetching project by ID: {e}")
        return None


def get_refined_question(user_question: str, project_data: dict) -> str:
    project_info_str = "\n".join([
        f"{key}: {value}" for key, value in project_data.items() if value is not None
    ])

    prompt = f"""
    당신은 데이터베이스 및 커리어 분석 전문가입니다.
    아래는 한 사용자의 프로젝트 이력 정보와 자연어 질문입니다.

    이 사용자의 이력을 반영하여, 사용자의 질문을 더 구체적이고 상황에 맞게 '하나의 질문'으로 재작성해 주세요.
    불필요한 일반화 없이, 사용자의 실제 경력과 질문이 연결되도록 하세요.

    [프로젝트 이력 정보]
    {project_info_str}

    [원래 사용자 질문]
    "{user_question}"

    [이력 기반으로 재작성된 질문]
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 커리어 기반 질문 최적화 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=300
    )

    refined_question = response.choices[0].message.content.strip()
    return refined_question

In [10]:
#제일 마지막 row 기준
project_id = "EMP-100014"
user_question = "내경험과연차를고려할때, 최종적인CareerGoal은어떤게있을까? 나와비슷한연차와경험을가진다른사람들은어떤선택을했는지알려줘"

project_data = fetch_project_by_id_sql(project_id)
print(project_data)
if project_data:
    prompt = get_refined_question(user_question, project_data)
    print(prompt)
else:
    print("❌ 해당 ID의 프로젝트 정보를 찾을 수 없습니다.")


{'profileId': 52, 'ID': 'EMP-100014', 'StartYear': 1.0, 'EndYear': 5.0, 'Project': '반도체 설비관리 및 설비제어, SHE 통합시스템 구축', 'ProjectScale': 2.0, 'Roles': 'PL, PM', 'SkillSet1': '자동화 Eng..', 'SkillSet2': 'nan', 'SkillSet3': 'nan', 'SkillSet4': 'nan', 'Industry': '제조', 'CareerImpact': None, 'CareerImpactDesc': 'nan'}
"반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 5년차 경력으로써, 비슷한 경력과 경험을 가진 다른 전문가들이 경력 발전을 위해 어떤 경로를 선택했는지 알려주세요."


### Text to SQL

### 1. original code

In [76]:
# from typing import List
# def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
#     schema = """
#     CREATE TABLE `project` (
#       `ID` text DEFAULT NULL,
#       `StartYear` double DEFAULT NULL,
#       `EndYear` double DEFAULT NULL,
#       `Project` text DEFAULT NULL,
#       `ProjectScale` double DEFAULT NULL,
#       `Roles` text DEFAULT NULL,
#       `SkillSet1` text DEFAULT NULL,
#       `SkillSet2` text DEFAULT NULL,
#       `SkillSet3` text DEFAULT NULL,
#       `SkillSet4` text DEFAULT NULL,
#       `Industry` text DEFAULT NULL,
#       `CareerImpact` double DEFAULT NULL
#     ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
#     """

#     roles = """
#     사용 가능한 Roles 값은 다음과 같습니다:
#     Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
#     Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
#     Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
#     TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
#     """

#     skillsets = """
#     사용 가능한 Skill Set 값은 다음과 같습니다:
#     Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
#     ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
#     Middleware/Database Eng, Data Center Eng, Cyber Security,
#     Application Architect, Data Architect, Technical Architect,
#     Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
#     AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
#     Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
#     Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
#     Stakeholder Mgmt, Governance & Public Mgmt,
#     Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
#     """

#     # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
#     intent_expand_prompt = f"""
#     당신은 데이터베이스 및 업무 도메인 전문가입니다.
#     아래 사용자의 질문에 대해,  
#     관련되거나 확장 가능한 여러 개의 의미 있는 질문(의도)를 생성해 주세요.  
#     가능한 경우 아래와 같이 리스트 형식으로 출력하세요(최대 3개):  
#     - 질문 1  
#     - 질문 2  
#     - 질문 3  
#     ...

#     질문: "{natural_language_query}"
#     """

#     intent_response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
#             {"role": "user", "content": intent_expand_prompt}
#         ],
#         temperature=0,
#         max_tokens=400
#     )

#     expanded_questions_text = intent_response.choices[0].message.content.strip()
#     expanded_questions = [
#         line.lstrip("- ").strip()
#         for line in expanded_questions_text.splitlines()
#         if line.startswith("- ")
#     ]
#     print(expanded_questions)

#     sql_queries = []

#     # 2단계: 확장된 질문 별로 SQL 생성
#     for q in expanded_questions:
#         sql_prompt = f"""
#         아래는 데이터베이스 스키마입니다:
#         {schema}

#         Roles 목록:
#         {roles}

#         Skill Set 목록:
#         {skillsets}

#         위 내용을 참고하여, 다음 질문에 적합한 SQL 쿼리를 작성해 주세요.
#         항상 'ID' 컬럼을 포함하여 조회하는 쿼리를 작성해 주세요.
        
#         질문: {q}

#         SQL 쿼리:
#         """

#         sql_response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "system", "content": "당신은 SQL 전문가입니다."},
#                 {"role": "user", "content": sql_prompt}
#             ],
#             max_tokens=300,
#             temperature=0,
#         )

#         sql_text = sql_response.choices[0].message.content.strip()
#         if sql_text.startswith("```sql"):
#             sql_text = sql_text.replace("```sql", "").replace("```", "").strip()
#         sql_queries.append(sql_text)

#     return sql_queries
    


### 2. 1개의 llm에서 확장질문, sql문 둘다 생성

In [77]:
# from typing import List
# import re

# def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
#     schema = """
#     CREATE TABLE `project` (
#     `ID` text DEFAULT NULL,
#     `StartYear` double DEFAULT NULL,
#     `EndYear` double DEFAULT NULL,
#     `Project` text DEFAULT NULL,
#     `ProjectScale` double DEFAULT NULL,
#     `Roles` text DEFAULT NULL,
#     `SkillSet1` text DEFAULT NULL,
#     `SkillSet2` text DEFAULT NULL,
#     `SkillSet3` text DEFAULT NULL,
#     `SkillSet4` text DEFAULT NULL,
#     `Industry` text DEFAULT NULL,
#     `CareerImpact` double DEFAULT NULL
#     ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
#     """

#     roles = """
#     사용 가능한 Roles 값은 다음과 같습니다:
#     Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
#     Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
#     Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
#     TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
#     """

#     skillsets = """
#     사용 가능한 Skill Set 값은 다음과 같습니다:
#     Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
#     ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
#     Middleware/Database Eng, Data Center Eng, Cyber Security,
#     Application Architect, Data Architect, Technical Architect,
#     Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
#     AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
#     Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
#     Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
#     Stakeholder Mgmt, Governance & Public Mgmt,
#     Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
#     """

#     # 한번의 LLM 호출로 질문 확장 + SQL 생성
#     combined_prompt = f"""
#     당신은 데이터베이스 및 업무 도메인 전문가입니다.

#     다음 작업을 수행해 주세요:
#     1. 사용자 질문에 대해 관련되거나 확장 가능한 의미 있는 질문 3개를 생성
#     2. 각 확장된 질문에 대해 간단하고 명확한 SQL 쿼리 작성 (항상 'ID' 컬럼 포함)

#     아래는 데이터베이스 스키마입니다:
#     {schema}

#     Roles 목록:
#     {roles}

#     Skill Set 목록:
#     {skillsets}
    
#     **SQL 작성 규칙**:
#     - 항상 'ID' 컬럼을 포함하세요
#     - 간단하고 실행 가능한 SQL만 작성하세요 
#     - 복잡한 CASE문이나 서브쿼리는 피하세요
#     - CareerImpact는 1(높은 영향도) 또는 0(낮은 영향도)만 사용하세요
#     - AND 조건은 반드시 필요한 경우에만 사용하고, 대부분의 필터링은 OR로 연결해 주세요.
    
#     정확히 아래 형식으로 출력해 주세요:
#     Q1: [간단한 확장 질문 1]
#     S1: SELECT ID [컬럼들] FROM project WHERE [조건];

#     Q2: [간단한 확장 질문 2]  
#     S2: SELECT ID [컬럼들] FROM project WHERE [조건];

#     Q3: [간단한 확장 질문 3]
#     S3: SELECT ID [컬럼들] FROM project WHERE [조건];

#     사용자 질문: "{natural_language_query}"
#     """

#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 간단하고 실용적인 SQL을 작성하는 전문가입니다."},
#             {"role": "user", "content": combined_prompt}
#         ],
#         temperature=0
#     )

#     content = response.choices[0].message.content.strip()
#     # print("=== LLM 응답 내용 ===")
#     # print(content)
#     # print("==================")
#     return content



    

In [78]:
import re

def extract_sql_queries(llm_response: str) -> list:
    print("=== 디버깅: 전체 응답 ===")
    print(llm_response)
    print("=" * 50)
    
    sql_queries = []
    
    # 간단한 방법: 줄별로 처리
    lines = llm_response.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('S') and ':' in line:
            print(f"찾은 S 라인: {line}")
            # S1:, S2:, S3: 뒤의 모든 내용 가져오기
            sql_part = line.split(':', 1)[1].strip()
            if sql_part and 'SELECT' in sql_part.upper():
                sql_queries.append(sql_part)
    
    # 만약 위 방법으로 안 되면 정규표현식 사용
    if len(sql_queries) < 3:
        print("정규표현식으로 재시도...")
        pattern = r'S\d+:\s*(SELECT.*?)(?=\s*Q\d+:|\s*S\d+:|\s*$)'
        matches = re.findall(pattern, llm_response, re.DOTALL | re.IGNORECASE)
        
        for match in matches:
            clean_sql = ' '.join(match.strip().split())
            if clean_sql:
                sql_queries.append(clean_sql)
    
    # print(f"=== 최종 추출된 SQL 개수: {len(sql_queries)} ===")
    # for i, sql in enumerate(sql_queries):
    #     print(f"SQL {i+1}: {sql[:100]}...")
    
    return sql_queries

In [79]:
print(prompt)
sqls = text_to_sql_with_intent_expansion(prompt)
print(sqls)

"저는 반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 경력이 있습니다. 이와 유사한 경험을 가진 사람들이 어떤 커리어 선택을 했는지 알려주실 수 있을까요?"
['반도체 산업에서의 경력을 가진 사람들 중에서 PL 및 PM 역할을 수행한 사람들의 프로젝트 규모(ProjectScale)는 어떤 범위에 해당하는지 알려주세요.', 'SHE 통합시스템 구축 프로젝트에 참여한 사람들 중에서 향후 커리어 발전을 위해 어떤 역량(SkillSet)을 더 강화하는 경향이 있는지 조사해볼 수 있을까요?', '반도체 설비제어 분야에서 PL 및 PM 역할을 수행한 경력이 있는 사람들 중에서 향후 이직 시 가장 선호하는 업무 영역(Industry)은 무엇인지 알려주세요.']
["SELECT ID, ProjectScale FROM project WHERE Industry = '반도체' AND Roles IN ('PL', 'PM');", "SELECT ID, SkillSet1, SkillSet2, SkillSet3, SkillSet4 FROM project WHERE Project LIKE '%SHE 통합시스템 구축%' AND SkillSet1 IN ('Front-end Dev', 'Back-end Dev', 'Mobile Dev', 'Cyber Security', 'Data Architect');", "SELECT ID, Industry FROM project WHERE Industry IN ('반도체', '제조') AND Roles IN ('PL', 'PM') ORDER BY CareerImpact DESC;"]


### 3. 질문생성, sql문 생성 llm 나눈뒤 industry placeholder에 추가

In [80]:
# from typing import List

# def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
#     schema = """
#     CREATE TABLE `project` (
#       `ID` text DEFAULT NULL,
#       `StartYear` double DEFAULT NULL,
#       `EndYear` double DEFAULT NULL,
#       `Project` text DEFAULT NULL,
#       `ProjectScale` double DEFAULT NULL,
#       `Roles` text DEFAULT NULL,
#       `SkillSet1` text DEFAULT NULL,
#       `SkillSet2` text DEFAULT NULL,
#       `SkillSet3` text DEFAULT NULL,
#       `SkillSet4` text DEFAULT NULL,
#       `Industry` text DEFAULT NULL,
#       `CareerImpact` double DEFAULT NULL
#     ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
#     """

#     roles = """
#     사용 가능한 Roles 값은 다음과 같습니다:
#     Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
#     Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
#     Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
#     TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
#     """

#     skillsets = """
#     사용 가능한 Skill Set 값은 다음과 같습니다:
#     Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
#     ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
#     Middleware/Database Eng, Data Center Eng, Cyber Security,
#     Application Architect, Data Architect, Technical Architect,
#     Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
#     AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
#     Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
#     Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
#     Stakeholder Mgmt, Governance & Public Mgmt,
#     Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
#     """

#     Industry = """
#     사용 가능한 Industry 값은 다음과 같습니다:
#     물류, 제2금융, 제조, 공공, 미디어, 통신, 금융, 공통, Global, 
#     대외, 제1금융, 은행, 의료, 유통, 보험, SK그룹, 유통/서비스, 유통/물류/서비스
#     """

#     # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
#     intent_expand_prompt = f"""
#     아래 사용자 질문을 데이터베이스 조회가 가능한 구체적인 질문 3개로 확장해 주세요.
#     각 질문은 역할, 기술스택, 산업, 프로젝트 정보로 필터링 가능해야 합니다.

#     원본 질문: "{natural_language_query}"

#     확장 시 고려사항:
#     - 역할(Roles), 기술스택(SkillSet), 산업(Industry), 프로젝트 규모, 경력 연차 등으로 필터링 가능한 질문

#     형식:
#     - 질문 1
#     - 질문 2  
#     - 질문 3
#     """

#     intent_response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
#             {"role": "user", "content": intent_expand_prompt}
#         ],
#         temperature=0
#     )

#     expanded_questions_text = intent_response.choices[0].message.content.strip()
#     expanded_questions = [
#         line.lstrip("- ").strip()
#         for line in expanded_questions_text.splitlines()
#         if line.startswith("- ")
#     ]
#     print(expanded_questions)

#     # 2단계: 모든 확장된 질문을 한 번에 SQL로 변환
#     # 확장된 질문들을 번호와 함께 포맷팅
#     # questions_formatted = "\n".join([f"{i+1}. {q}" for i, q in enumerate(expanded_questions)])
    
#     combined_sql_prompt = f"""
#     **경고: 아래 목록에 정확히 있는 값만 사용하세요. 추측하거나 새로운 값을 만들지 마세요.**

#     스키마: {schema}

#     **사용 가능한 Roles (정확히 이것만 사용):**
#     {roles}

#     **사용 가능한 SkillSets (정확히 이것만 사용):**
#     {skillsets}

#     **사용 가능한 Industry (정확히 이것만 사용):**
#     {Industry}

#     **절대 금지사항:**
#     - Python, C++, PLC, SHE 통합시스템 구축, 기술 동향, 자동차, 전자제품 등 위 목록에 없는 값 사용 금지
#     - 새로운 값 생성 금지
#     - 추측 금지

#     **필수 규칙:**
#     - ID 컬럼 포함
#     - 위 목록의 정확한 값만 사용
#     - OR 조건 위주로 작성

#     질문들: {questions_formatted}

#     응답:
#     S1: SELECT ID, [컬럼] FROM project WHERE [위 목록의 정확한 값만 사용];
#     S2: SELECT ID, [컬럼] FROM project WHERE [위 목록의 정확한 값만 사용];
#     S3: SELECT ID, [컬럼] FROM project WHERE [위 목록의 정확한 값만 사용];
#     """

#     sql_response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 SQL 전문가입니다."},
#             {"role": "user", "content": combined_sql_prompt}
#         ],
#         temperature=0
#     )

#     # 응답에서 SQL 쿼리들을 파싱
#     sql_response_text = sql_response.choices[0].message.content.strip()
#     print('='*50)
#     print(sql_response_text)
#     print("진짜 3개의 쿼리를 생성하는지 확인해보자")

#     sql_queries = []
    
#     # Query 1:, Query 2:, Query 3: 패턴으로 분리
#     import re
#     query_pattern = r'Query \d+:\s*\n(.*?)(?=Query \d+:|$)'
#     matches = re.findall(query_pattern, sql_response_text, re.DOTALL)
    
#     for match in matches:
#         sql_text = match.strip()
#         # ```sql 태그 제거
#         if sql_text.startswith("```sql"):
#             sql_text = sql_text.replace("```sql", "").replace("```", "").strip()
#         elif sql_text.startswith("```"):
#             sql_text = sql_text.replace("```", "").strip()
        
#         # 줄바꿈 문자를 공백으로 대체하여 한 줄로 만들기
#         sql_text = " ".join(sql_text.split())
        
#         if sql_text:
#             sql_queries.append(sql_text)
    
#     # 만약 정규식으로 파싱이 실패했다면, 대안 방법 사용
#     if not sql_queries:
#         lines = sql_response_text.split('\n')
#         current_query = ""
#         for line in lines:
#             if line.strip().startswith("SELECT") or current_query:
#                 current_query += line + " "
#                 if line.strip().endswith(";") or (line.strip() and not line.strip().startswith("SELECT") and current_query.count("SELECT") > 0):
#                     # 줄바꿈 문자를 공백으로 대체하여 한 줄로 만들기
#                     cleaned_query = " ".join(current_query.strip().split())
#                     sql_queries.append(cleaned_query)
#                     current_query = ""

#     return sql_queries

### 4. 아래는 데이터 찾기 너무 부적합한 확장 질문들이 생성됨

In [81]:
# from typing import List

# def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
#     schema = """
#     CREATE TABLE `project` (
#       `ID` text DEFAULT NULL,
#       `StartYear` double DEFAULT NULL,
#       `EndYear` double DEFAULT NULL,
#       `Project` text DEFAULT NULL,
#       `ProjectScale` double DEFAULT NULL,
#       `Roles` text DEFAULT NULL,
#       `SkillSet1` text DEFAULT NULL,
#       `SkillSet2` text DEFAULT NULL,
#       `SkillSet3` text DEFAULT NULL,
#       `SkillSet4` text DEFAULT NULL,
#       `Industry` text DEFAULT NULL,
#       `CareerImpact` double DEFAULT NULL
#     ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
#     """

#     roles = """
#     사용 가능한 Roles 값은 다음과 같습니다:
#     Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
#     Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
#     Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
#     TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
#     """

#     skillsets = """
#     사용 가능한 Skill Set 값은 다음과 같습니다:
#     Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
#     ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
#     Middleware/Database Eng, Data Center Eng, Cyber Security,
#     Application Architect, Data Architect, Technical Architect,
#     Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
#     AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
#     Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
#     Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
#     Stakeholder Mgmt, Governance & Public Mgmt,
#     Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
#     """

#     # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
#     intent_expand_prompt = f"""
#     당신은 데이터베이스 및 업무 도메인 전문가입니다.
#     사용자 질문에 대해 관련되거나 확장 가능한 의미 있는 간단한 확장 질문 3개를 생성해주세요.

#     중요한 규칙:
#     1. 원본 질문의 핵심 키워드를 포함하되, 다양한 각도에서 데이터를 조회할 수 있도록 확장
#     2. 되도록 위에 제공된 Roles, Skill Set, Industry 목록의 값들을 사용할 것
#     3. 데이터베이스 필터링 가능한 조건만 사용 (Roles, SkillSet1-4, Project, ProjectScale, Industry, CareerImpact, StartYear, EndYear 등)
#     4. 추상적이거나 감정적인 내용은 절대 포함하지 말 것
#     5. 각 질문은 역할, 기술스택, 산업, 프로젝트 정보로 필터링 가능해야 합니다.
    
#     가능한 경우 아래와 같이 리스트 형식으로 출력하세요(최대 3개):  
#     - 질문 1  
#     - 질문 2  
#     - 질문 3  
#     ...

#     질문: "{natural_language_query}"
#     """

#     intent_response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
#             {"role": "user", "content": intent_expand_prompt}
#         ],
#         temperature=0
#     )

#     expanded_questions_text = intent_response.choices[0].message.content.strip()
#     expanded_questions = [
#         line.lstrip("- ").strip()
#         for line in expanded_questions_text.splitlines()
#         if line.startswith("- ")
#     ]
#     print(expanded_questions)

#     # 2단계: 확장된 질문들을 한 번에 SQL로 변환
#     # 질문들을 placeholder로 만들어서 한 번의 호출로 처리
#     questions_placeholder = "\n".join([f"{i+1}. {q}" for i, q in enumerate(expanded_questions)])
    
#     batch_sql_prompt = f"""
#     아래는 데이터베이스 스키마입니다:
#     {schema}

#     Roles 목록:
#     {roles}

#     Skill Set 목록:
#     {skillsets}

#     위 내용을 참고하여, 다음 질문들에 각각 적합한 SQL 쿼리를 작성해 주세요.
    
#     SQL 작성 가이드라인:
#     1. 항상 'ID' 컬럼을 포함하여 조회하는 쿼리를 작성해 주세요.
#     2. 가능한 많은 관련 데이터를 가져오기 위해 OR 조건을 적극 활용해 주세요.
#     3. AND 조건은 반드시 필요한 경우에만 사용하고, 대부분의 필터링은 OR로 연결해 주세요.
#     4. LIKE 연산자를 사용할 때는 부분 매칭(%키워드%)을 활용해 주세요.
#     5. 숫자 조건(CareerImpact, ProjectScale 등)은 범위 조건(>=, <=)을 사용해 주세요.
#     6. 간단하고 실행 가능한 SQL만 작성하세요 
    
#     각 질문에 대해 다음 형식으로 응답해 주세요:
    
#     질문 1에 대한 SQL:
#     ```sql
#     [SQL 쿼리]
#     ```
    
#     질문 2에 대한 SQL:
#     ```sql
#     [SQL 쿼리]
#     ```
    
#     질문 3에 대한 SQL:
#     ```sql
#     [SQL 쿼리]
#     ```
    
#     질문들:
#     {questions_placeholder}
#     """

#     sql_response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "당신은 SQL 전문가입니다. 여러 질문에 대해 각각의 SQL 쿼리를 생성해주세요."},
#             {"role": "user", "content": batch_sql_prompt}
#         ],  
#         temperature=0,
#     )

#     # 응답에서 SQL 쿼리들 추출
#     response_text = sql_response.choices[0].message.content.strip()
    
#     # SQL 블록들을 찾아서 추출
#     sql_queries = []
#     import re
    
#     # ```sql과 ``` 사이의 내용을 찾는 정규표현식
#     sql_blocks = re.findall(r'```sql\s*(.*?)\s*```', response_text, re.DOTALL)
    
#     for sql_block in sql_blocks:
#         # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
#         cleaned_sql = ' '.join(sql_block.strip().split())
#         sql_queries.append(cleaned_sql)
    
#     # 만약 정규표현식으로 찾지 못했다면 다른 방법으로 시도
#     if not sql_queries:
#         lines = response_text.split('\n')
#         current_sql = []
#         in_sql_block = False
        
#         for line in lines:
#             if '```sql' in line.lower():
#                 in_sql_block = True
#                 continue
#             elif '```' in line and in_sql_block:
#                 if current_sql:
#                     # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
#                     cleaned_sql = ' '.join(' '.join(current_sql).strip().split())
#                     sql_queries.append(cleaned_sql)
#                     current_sql = []
#                 in_sql_block = False
#             elif in_sql_block:
#                 current_sql.append(line)
    
#     return sql_queries

In [82]:
print(prompt)
sqls = text_to_sql_with_intent_expansion(prompt)
for i in sqls:
    print(i)

"저는 반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 경력이 있습니다. 이와 유사한 경험을 가진 사람들이 어떤 커리어 선택을 했는지 알려주실 수 있을까요?"
['반도체 산업에서의 경력을 가진 사람들 중에서 현재 어떤 프로젝트에 참여하고 있는지 알려주세요.', '반도체 설비제어 분야에서 PL 및 PM 역할을 수행한 사람들 중에서 어떤 산업군에서 활동하고 있는지 조사해볼 수 있을까요?', 'SHE 통합시스템 구축 프로젝트 경험이 있는 사람들 중에서 어떤 스킬셋이 가장 많이 요구되는지 알려주세요.']
SELECT ID, Project FROM project WHERE Industry = '반도체' AND Roles LIKE '%Engineer%' OR Roles LIKE '%Dev%' OR Roles LIKE '%Architect%'
SELECT ID, Industry FROM project WHERE Roles IN ('PL', 'PM') AND SkillSet1 LIKE '%ERP%' OR SkillSet1 LIKE '%Factory%' OR SkillSet1 LIKE '%Automation%'
SELECT ID, SkillSet1, SkillSet2, SkillSet3, SkillSet4 FROM project WHERE Project LIKE '%SHE%' AND SkillSet1 IN ('Cyber Security', 'Data Architect', 'Technical Architect')


### 5. 확장질문 수정 - 이게 젤 났다

In [94]:
from typing import List

def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
    schema = """
    CREATE TABLE `project` (
      `ID` text DEFAULT NULL,
      `StartYear` double DEFAULT NULL,
      `EndYear` double DEFAULT NULL,
      `Project` text DEFAULT NULL,
      `ProjectScale` double DEFAULT NULL,
      `Roles` text DEFAULT NULL,
      `SkillSet1` text DEFAULT NULL,
      `SkillSet2` text DEFAULT NULL,
      `SkillSet3` text DEFAULT NULL,
      `SkillSet4` text DEFAULT NULL,
      `Industry` text DEFAULT NULL,
      `CareerImpact` double DEFAULT NULL
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    roles = """
    사용 가능한 Roles 값은 다음과 같습니다:
    Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
    Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
    Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
    TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
    """

    skillsets = """
    사용 가능한 Skill Set 값은 다음과 같습니다:
    Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
    ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
    Middleware/Database Eng, Data Center Eng, Cyber Security,
    Application Architect, Data Architect, Technical Architect,
    Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
    AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
    Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
    Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
    Stakeholder Mgmt, Governance & Public Mgmt,
    Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
    """

    Industry = """
    사용 가능한 Industry 값은 다음과 같습니다:
    물류, 제2금융, 제조, 공공, 미디어, 통신, 금융, 공통, Global, 
    대외, 제1금융, 은행, 의료, 유통, 보험, SK그룹, 유통/서비스, 유통/물류/서비스
    """

    # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
    intent_expand_prompt = f"""
    당신은 데이터베이스 및 업무 도메인 전문가입니다.
    아래 사용자 질문과 관련된 다양한 데이터를 찾을 수 있는 3개의 확장 질문을 만들어주세요.

    중요한 규칙:
    1. 원본 질문의 핵심 키워드를 포함하되, 다양한 각도에서 데이터를 조회할 수 있도록 확장
    3. 데이터베이스 필터링 가능한 조건만 사용 (Roles, SkillSet1-4, Project, ProjectScale, Industry, CareerImpact, StartYear, EndYear 등)
    4. 추상적이거나 감정적인 내용은 절대 포함하지 말 것

    확장 예시:
    - 원본 키워드를 유지하면서 다른 조건들과 조합
    - 원본보다 더 넓은 범위로 확장하여 관련 데이터 탐색
    - 원본의 일부 키워드를 다른 관련 키워드로 대체


    질문: "{natural_language_query}"

    아래 형식으로만 응답하세요:
    - 질문 1
    - 질문 2  
    - 질문 3
    """
    
    intent_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
            {"role": "user", "content": intent_expand_prompt}
        ],
        temperature=0
    )

    expanded_questions_text = intent_response.choices[0].message.content.strip()
    expanded_questions = [
        line.lstrip("- ").strip()
        for line in expanded_questions_text.splitlines()
        if line.startswith("- ")
    ]
    print(expanded_questions)

    # 2단계: 확장된 질문들을 한 번에 SQL로 변환
    # 질문들을 placeholder로 만들어서 한 번의 호출로 처리
    questions_placeholder = "\n".join([f"{i+1}. {q}" for i, q in enumerate(expanded_questions)])
    
    batch_sql_prompt = f"""
    아래는 데이터베이스 스키마입니다:
    {schema}

    Roles 목록:
    {roles}

    Skill Set 목록:
    {skillsets}

    Industry 목록:
    {Industry}

    위 내용을 참고하여, 다음 질문들에 각각 적합한 SQL 쿼리를 작성해 주세요.
    
    SQL 작성 가이드라인:
    1. 항상 'ID' 컬럼을 포함하여 조회하는 쿼리를 작성해 주세요.
    2. 가능한 많은 관련 데이터를 가져오기 위해 OR 조건을 적극 활용해 주세요.
    3. AND 조건은 반드시 필요한 경우에만 사용하고, 대부분의 필터링은 OR로 연결해 주세요.
    4. LIKE 연산자를 사용할 때는 부분 매칭(%키워드%)을 활용해 주세요.
    5. 숫자 조건(CareerImpact, ProjectScale 등)은 범위 조건(>=, <=)을 사용해 주세요.
    6. 간단하고 실행 가능한 SQL만 작성하세요 
    7. schema, roles, skillsets, industry 사용시 목록에 있는 값만 사용하세요. 되도록 추측하거나 새로운 값을 만들지 마세요.
    
    각 질문에 대해 다음 형식으로 응답해 주세요:
    
    질문 1에 대한 SQL:
    ```sql
    [SQL 쿼리]
    ```
    
    질문 2에 대한 SQL:
    ```sql
    [SQL 쿼리]
    ```
    
    질문 3에 대한 SQL:
    ```sql
    [SQL 쿼리]
    ```
    
    질문들:
    {questions_placeholder}
    """

    sql_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 SQL 전문가입니다. 여러 질문에 대해 각각의 SQL 쿼리를 생성해주세요."},
            {"role": "user", "content": batch_sql_prompt}
        ],  
        temperature=0,
    )

    # 응답에서 SQL 쿼리들 추출
    response_text = sql_response.choices[0].message.content.strip()
    
    # SQL 블록들을 찾아서 추출
    sql_queries = []
    import re
    
    # ```sql과 ``` 사이의 내용을 찾는 정규표현식
    sql_blocks = re.findall(r'```sql\s*(.*?)\s*```', response_text, re.DOTALL)
    
    for sql_block in sql_blocks:
        # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
        cleaned_sql = ' '.join(sql_block.strip().split())
        sql_queries.append(cleaned_sql)
    
    # 만약 정규표현식으로 찾지 못했다면 다른 방법으로 시도
    if not sql_queries:
        lines = response_text.split('\n')
        current_sql = []
        in_sql_block = False
        
        for line in lines:
            if '```sql' in line.lower():
                in_sql_block = True
                continue
            elif '```' in line and in_sql_block:
                if current_sql:
                    # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
                    cleaned_sql = ' '.join(' '.join(current_sql).strip().split())
                    sql_queries.append(cleaned_sql)
                    current_sql = []
                in_sql_block = False
            elif in_sql_block:
                current_sql.append(line)
    
    return sql_queries

In [95]:
print(prompt)
sqls = text_to_sql_with_intent_expansion(prompt)
for i in sqls:
    print(i)

"저는 반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 경력이 있습니다. 이와 유사한 경험을 가진 사람들이 어떤 커리어 선택을 했는지 알려주실 수 있을까요?"
['반도체 산업에서의 경력을 가진 사람들 중에서, 설비제어 및 설비관리 역할을 수행한 사람들이 참여한 프로젝트의 규모는 어떠한가요?', 'SHE 통합시스템 구축 프로젝트에 참여한 PL 및 PM 역할을 수행한 사람들 중에서, 다양한 산업 분야로 이직한 사람들이 어떤 역할을 맡고 있는지 알 수 있을까요?', '설비제어 및 설비관리 분야에서 경력을 쌓은 후에 커리어에 어떠한 영향을 미친 사람들이 주로 어떤 산업 분야로 이직하는 경향이 있나요?']
SELECT ID, ProjectScale FROM project WHERE (Roles LIKE '%Factory%' OR Roles LIKE '%Automation%') AND Industry = '반도체'
SELECT ID, Roles FROM project WHERE (Roles LIKE '%PL%' OR Roles LIKE '%PM%') AND Project LIKE '%SHE%'
SELECT ID, Industry FROM project WHERE (SkillSet1 LIKE '%Factory%' OR SkillSet2 LIKE '%Factory%' OR SkillSet3 LIKE '%Factory%' OR SkillSet4 LIKE '%Factory%') AND CareerImpact >= 3


### 6.ori 프롬프트 적용

In [13]:
from typing import List

def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
    schema = """
    CREATE TABLE `project` (
      `ID` text DEFAULT NULL,
      `StartYear` double DEFAULT NULL,
      `EndYear` double DEFAULT NULL,
      `Project` text DEFAULT NULL,
      `ProjectScale` double DEFAULT NULL,
      `Roles` text DEFAULT NULL,
      `SkillSet1` text DEFAULT NULL,
      `SkillSet2` text DEFAULT NULL,
      `SkillSet3` text DEFAULT NULL,
      `SkillSet4` text DEFAULT NULL,
      `Industry` text DEFAULT NULL,
      `CareerImpact` double DEFAULT NULL
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    roles = """
    사용 가능한 Roles 값은 다음과 같습니다:
    Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
    Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
    Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
    TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
    """

    skillsets = """
    사용 가능한 Skill Set 값은 다음과 같습니다:
    Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
    ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
    Middleware/Database Eng, Data Center Eng, Cyber Security,
    Application Architect, Data Architect, Technical Architect,
    Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
    AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
    Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
    Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
    Stakeholder Mgmt, Governance & Public Mgmt,
    Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
    """

    Industry = """
    사용 가능한 Industry 값은 다음과 같습니다:
    물류, 제2금융, 제조, 공공, 미디어, 통신, 금융, 공통, Global, 
    대외, 제1금융, 은행, 의료, 유통, 보험, SK그룹, 유통/서비스, 유통/물류/서비스
    """

    # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
    intent_expand_prompt = f"""
    당신은 데이터베이스 및 업무 도메인 전문가입니다.
    아래 사용자의 질문에 대해,  
    관련되거나 확장 가능한 여러 개의 의미 있는 질문(의도)를 생성해 주세요.  
    가능한 경우 아래와 같이 리스트 형식으로 출력하세요(최대 3개):  
    - 질문 1  
    - 질문 2  
    - 질문 3  
    ...

    질문: "{natural_language_query}"
    """
    
    intent_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
            {"role": "user", "content": intent_expand_prompt}
        ],
        temperature=0
    )

    expanded_questions_text = intent_response.choices[0].message.content.strip()
    expanded_questions = [
        line.lstrip("- ").strip()
        for line in expanded_questions_text.splitlines()
        if line.startswith("- ")
    ]
    print(expanded_questions)

    # 2단계: 확장된 질문들을 한 번에 SQL로 변환
    # 질문들을 placeholder로 만들어서 한 번의 호출로 처리
    questions_placeholder = "\n".join([f"{i+1}. {q}" for i, q in enumerate(expanded_questions)])
    print("질문 잘 들어가고 있니")
    print(questions_placeholder)
    print("="*50)
    
    batch_sql_prompt = f"""
        아래는 데이터베이스 스키마입니다:
        {schema}

        Roles 목록:
        {roles}

        Skill Set 목록:
        {skillsets}

        위 내용을 참고하여, 다음 질문에 적합한 SQL 쿼리를 작성해 주세요.
        항상 'ID' 컬럼을 포함하여 조회하는 쿼리를 작성해 주세요.
        
        질문: {questions_placeholder}

        SQL 쿼리:
        """

    sql_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 SQL 전문가입니다. 여러 질문에 대해 각각의 SQL 쿼리를 생성해주세요."},
            {"role": "user", "content": batch_sql_prompt}
        ],  
        temperature=0,
    )

    # 응답에서 SQL 쿼리들 추출
    response_text = sql_response.choices[0].message.content.strip()
    
    # SQL 블록들을 찾아서 추출
    sql_queries = []
    import re
    
    # ```sql과 ``` 사이의 내용을 찾는 정규표현식
    sql_blocks = re.findall(r'```sql\s*(.*?)\s*```', response_text, re.DOTALL)
    
    for sql_block in sql_blocks:
        # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
        cleaned_sql = ' '.join(sql_block.strip().split())
        sql_queries.append(cleaned_sql)
    
    # 만약 정규표현식으로 찾지 못했다면 다른 방법으로 시도
    if not sql_queries:
        lines = response_text.split('\n')
        current_sql = []
        in_sql_block = False
        
        for line in lines:
            if '```sql' in line.lower():
                in_sql_block = True
                continue
            elif '```' in line and in_sql_block:
                if current_sql:
                    # 개행문자를 공백으로 변환하고 여러 공백을 하나로 정리
                    cleaned_sql = ' '.join(' '.join(current_sql).strip().split())
                    sql_queries.append(cleaned_sql)
                    current_sql = []
                in_sql_block = False
            elif in_sql_block:
                current_sql.append(line)
    
    return sql_queries

In [14]:
print(prompt)
sqls = text_to_sql_with_intent_expansion(prompt)
for i in sqls:
    print(i)

"반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 5년차 경력으로써, 비슷한 경력과 경험을 가진 다른 전문가들이 경력 발전을 위해 어떤 경로를 선택했는지 알려주세요."
['다른 전문가들이 경력 발전을 위해 선택한 교육 및 자격증 프로그램은 무엇인가요?', '비슷한 경력과 경험을 가진 전문가들이 새로운 기술 및 도구 습득을 위해 어떤 노력을 기울였는지 알 수 있을까요?', '경력 발전을 위해 다른 전문가들이 어떤 산업 혹은 분야로 전환했는지 알고 계신가요?']
질문 잘 들어가고 있니
1. 다른 전문가들이 경력 발전을 위해 선택한 교육 및 자격증 프로그램은 무엇인가요?
2. 비슷한 경력과 경험을 가진 전문가들이 새로운 기술 및 도구 습득을 위해 어떤 노력을 기울였는지 알 수 있을까요?
3. 경력 발전을 위해 다른 전문가들이 어떤 산업 혹은 분야로 전환했는지 알고 계신가요?
SELECT ID, SkillSet1, SkillSet2, SkillSet3, SkillSet4 FROM project WHERE SkillSet1 IN ( 'Front-end Dev', 'Back-end Dev', 'Mobile Dev', 'Factory 기획/설계', '자동화 Eng', '지능화 Eng', 'ERP_FCM', 'ERP_SCM', 'ERP_HCM', 'ERP_T&E', 'Biz. Solution', 'System/Network Eng', 'Middleware/Database Eng', 'Data Center Eng', 'Cyber Security', 'Application Architect', 'Data Architect', 'Technical Architect', 'Infra PM', 'Application PM', 'Solution PM', 'PMO', 'Quality Eng', 'Offshoring Service Professional', 'AI/Data Dev', 'Generative AI Dev', 'Generative A

In [99]:
from collections import Counter

all_ids = []

for i, sql in enumerate(sqls, 1):
    print(f"\n▶ Executing SQL {i}:\n{sql}\n")
    try:
        result = execute_sql(sql)
        if isinstance(result, tuple):
            columns, rows = result
            print("✅ Query Result Columns:")
            print(columns)
            print("✅ Query Result Rows:")
            for row in rows:
                print(row)

            # columns를 리스트로 변환 후 ID 위치 찾기
            try:
                id_index = list(columns).index('ID')
            except ValueError:
                print("❌ 'ID' 컬럼이 결과에 없습니다.")
                continue

            # 이번 쿼리 결과에서 ID 리스트 추출해 all_ids에 누적
            all_ids.extend([row[id_index] for row in rows if row[id_index] is not None])

        else:
            print(f"✅ Affected rows: {result}")

    except Exception as e:
        print(f"❌ Error executing SQL {i}: {e}")

# 모든 쿼리 결과 통합 후 가장 많이 나온 ID 3개 출력
counter = Counter(all_ids)
most_common_ids = counter.most_common(3)

print("\n⭐ 전체 SQL 결과에서 조회된 ID 중 상위 3개:")
for id_val, count in most_common_ids:
    print(f"ID: {id_val}, Count: {count}")



▶ Executing SQL 1:
SELECT ID, ProjectScale FROM project WHERE (Roles LIKE '%Factory%' OR Roles LIKE '%Automation%') AND Industry = '반도체'

✅ Query Result Columns:
RMKeyView(['ID', 'ProjectScale'])
✅ Query Result Rows:

▶ Executing SQL 2:
SELECT ID, Roles FROM project WHERE (Roles LIKE '%PL%' OR Roles LIKE '%PM%') AND Project LIKE '%SHE%'

✅ Query Result Columns:
RMKeyView(['ID', 'Roles'])
✅ Query Result Rows:
('EMP-100014', 'PL, PM')
('EMP-010113', 'Application, Arch., Architect, Cloud, PL')

▶ Executing SQL 3:
SELECT ID, Industry FROM project WHERE (SkillSet1 LIKE '%Factory%' OR SkillSet2 LIKE '%Factory%' OR SkillSet3 LIKE '%Factory%' OR SkillSet4 LIKE '%Factory%') AND CareerImpact >= 3

✅ Query Result Columns:
RMKeyView(['ID', 'Industry'])
✅ Query Result Rows:

⭐ 전체 SQL 결과에서 조회된 ID 중 상위 3개:
ID: EMP-100014, Count: 1
ID: EMP-010113, Count: 1


In [100]:
# 상위 3개 ID를 추출해 SQL IN 조건으로 사용
top_ids = [id_val for id_val, _ in most_common_ids]

# IN 절을 위한 문자열 생성
in_clause = ", ".join(f"'{id_val}'" for id_val in top_ids)

# SQL 쿼리 생성
query = f"""
SELECT *
FROM project
WHERE ID IN ({in_clause});
"""

print("\n📌 상위 3개 ID에 해당하는 전체 프로젝트 이력:")
try:
    result = execute_sql(query)
    if isinstance(result, tuple):
        columns, rows = result
        
        # ID별로 그룹화
        from collections import defaultdict
        person_projects = defaultdict(list)
        
        for row in rows:
            row_dict = dict(zip(columns, row))
            person_id = row_dict.get('ID')
            person_projects[person_id].append(row_dict)
        
        # 각 사람별로 전체 이력 출력
        for person_id, projects in person_projects.items():
            print(f"\n{'=' * 80}")
            print(f"사번: {person_id} (총 {len(projects)}개 프로젝트)")
            print('=' * 80)
            
            # 프로젝트를 시작년도 순으로 정렬
            projects.sort(key=lambda x: x.get('StartYear', 0) or 0)
            
            for i, project in enumerate(projects, 1):
                # 연차 계산
                start_year = project.get('StartYear', 0) or 0
                end_year = project.get('EndYear', 0) or 0
                period = f"{int(start_year)}~{int(end_year)}" if start_year > 0 and end_year > 0 else "N/A"
                
                # 스킬셋 통합
                skills = []
                for skill_col in ['SkillSet1', 'SkillSet2', 'SkillSet3', 'SkillSet4']:
                    skill = project.get(skill_col)
                    if skill and skill.strip():
                        skills.append(skill.strip())
                skills_str = ", ".join(skills) if skills else "N/A"
                
                project_info = f"""
프로젝트 {i}:
기 간 : {period}
프로젝트 : {project.get('Project', 'N/A')}
역 할 : {project.get('Roles', 'N/A')}
스 킬 셋 : {skills_str}
프로젝트 규모 : {project.get('ProjectScale', 'N/A')}
도 메 인 : {project.get('Industry', 'N/A')}
커리어 임팩트 : {project.get('CareerImpact', 'N/A')}
"""
                print(project_info)
            
            # 종합 정보
            total_years = sum([(p.get('EndYear', 0) or 0) - (p.get('StartYear', 0) or 0) + 1 
                              for p in projects if p.get('StartYear') and p.get('EndYear')])
            all_skills = set()
            all_industries = set()
            all_roles = set()
            
            for p in projects:
                for skill_col in ['SkillSet1', 'SkillSet2', 'SkillSet3', 'SkillSet4']:
                    skill = p.get(skill_col)
                    if skill and skill.strip():
                        all_skills.add(skill.strip())
                
                if p.get('Industry'):
                    all_industries.add(p.get('Industry'))
                if p.get('Roles'):
                    all_roles.add(p.get('Roles'))
            
            summary = f"""
종합 경력 정보:
총 경력 : 약 {total_years}년 | 프로젝트 수: {len(projects)}개 | 주요 역할: {', '.join(all_roles)} | 
전체 스킬셋: {', '.join(all_skills)} | 경험 도메인: {', '.join(all_industries)}
"""
            print(summary)
    else:
        print(f"✅ Affected rows: {result}")
except Exception as e:
    print(f"❌ Error fetching project details: {e}")


📌 상위 3개 ID에 해당하는 전체 프로젝트 이력:

사번: EMP-100014 (총 1개 프로젝트)

프로젝트 1:
기 간 : 1~5
프로젝트 : 반도체 설비관리 및 설비제어, SHE 통합시스템 구축
역 할 : PL, PM
스 킬 셋 : 자동화 Eng.., nan, nan, nan
프로젝트 규모 : 2.0
도 메 인 : 제조
커리어 임팩트 : None


종합 경력 정보:
총 경력 : 약 5.0년 | 프로젝트 수: 1개 | 주요 역할: PL, PM | 
전체 스킬셋: nan, 자동화 Eng.. | 경험 도메인: 제조


사번: EMP-010113 (총 3개 프로젝트)

프로젝트 1:
기 간 : 1~2
프로젝트 : 반도체 설비관리 및 설비제어, SHE 통합시스템 구축
역 할 : Application, Arch., Architect, Cloud, PL
스 킬 셋 : ERP_SCM, Generative AI Dev., ERP_HCM, SCM
프로젝트 규모 : 5.0
도 메 인 : 통신
커리어 임팩트 : 0.0


프로젝트 2:
기 간 : 3~6
프로젝트 : IT서비스HV 차세대 영업 관리 시스템
역 할 : DB, 시스템, 어드민
스 킬 셋 : Generative AI Model Dev., Strategy Planning, ERP_FCM, AIX
프로젝트 규모 : 5.0
도 메 인 : 유통/물류/서비스
커리어 임팩트 : 1.0


프로젝트 3:
기 간 : 7~9
프로젝트 : 멀티 클라우드 기반 Global Connectivity 서비스 확장
역 할 : (System, Engineer), PL, SE
스 킬 셋 : ERP_HCM, AI/Data Dev., Sales, Sales
프로젝트 규모 : 4.0
도 메 인 : 유통/물류/서비스
커리어 임팩트 : 1.0


종합 경력 정보:
총 경력 : 약 9.0년 | 프로젝트 수: 3개 | 주요 역할: (System, Engineer), PL, SE, Application, Arch., Architect, Cloud, PL,

In [86]:
# 상위 3개 ID를 추출해 SQL IN 조건으로 사용
top_ids = [id_val for id_val, _ in most_common_ids]

# IN 절을 위한 문자열 생성
in_clause = ", ".join(f"'{id_val}'" for id_val in top_ids)

# SQL 쿼리 생성
query = f"""
SELECT *
FROM project
WHERE ID IN ({in_clause});
"""

# 실행
print("\n📌 상위 3개 ID에 해당하는 project 정보:")
try:
    result = execute_sql(query)
    if isinstance(result, tuple):
        columns, rows = result
        print("✅ Columns:")
        print(columns)
        print("✅ Rows:")
        for row in rows:
            print(row)
    else:
        print(f"✅ Affected rows: {result}")
except Exception as e:
    print(f"❌ Error fetching project details: {e}")



📌 상위 3개 ID에 해당하는 project 정보:
✅ Columns:
RMKeyView(['profileId', 'ID', 'StartYear', 'EndYear', 'Project', 'ProjectScale', 'Roles', 'SkillSet1', 'SkillSet2', 'SkillSet3', 'SkillSet4', 'Industry', 'CareerImpact', 'CareerImpactDesc'])
✅ Rows:
(66, 'EMP-010028', 1.0, 2.0, '제2금융권 IT인프라 통합유지보수', 4.0, 'R2R, 개발', 'Infra PM -- 대형PM', 'Financial Mgmt.', 'PMO', 'Quality Eng.', '미디어', 0.0, 'nan')
(66, 'EMP-010028', 3.0, 6.0, 'Appication Architect/온라인서비스 차세대, Appication Architect/제2금융권 생명 OS', 3.0, 'MPA, 개발', 'ESG/SHE', 'SCM', 'PMO', 'Domain Expert', '유통/물류/서비스', 0.0, 'nan')
(75, 'EMP-010037', 1.0, 3.0, '통신사 G2(차세대) 구축', 4.0, 'AA, 팀장', 'Infra PM -- 대형PM', 'Offshoring Service Professional', 'Solution PM -- 대형PM', 'ERP_HCM', '의료', 0.0, 'nan')
(75, 'EMP-010037', 4.0, 5.0, '제2금융 Vitality 시스템 구축 및 운영', 5.0, '수행PM', 'Mobile Dev.', '자동화 Eng.', 'SCM', 'Generative AI Model Dev.', 'Global', 1.0, 'IT 개발자 및 NMS에 대한 전반적인 구조, 환경등에 대한 이해도 향상')
(75, 'EMP-010037', 6.0, 7.0, '제2금융 IDC 이전 및 통계인프라 시스템 구축', 4.0, '공통PL,

In [87]:
def rows_to_text(columns, rows) -> str:
    result = []
    for i, row in enumerate(rows, 1):
        row_dict = dict(zip(columns, row))
        row_str = "\n".join([f"{key}: {value}" for key, value in row_dict.items() if value is not None])
        result.append(f"🔹 프로젝트 {i}\n{row_str}")
    return "\n\n".join(result)

def generate_llm_response_from_projects(columns, rows, prompt: str) -> str:
    project_info_text = rows_to_text(columns, rows)

    full_prompt = f"""
    당신은 커리어 분석 전문가입니다.

    아래는 한 **조직 구성원**과 **질문자**의 프로젝트 및 경력 이력 정보입니다.  
    이 정보를 참고하여, 사용자 질문에 응답해 주세요.

    [1] 조직 구성원 이력 정보
    {project_info_text}

    [2] 질문자의 프로젝트 정보
    {project_data}

    아래 요청에 응답해 주세요:

    - 조직 구성원의 경력에서 중요한 역할(Role), 기술(SkillSet), 산업(Industry), 커리어 임팩트(CareerImpact) 등을 분석해 주세요.  
    - 질문자의 경력과 비교했을 때, 어떤 공통점 또는 차이점이 있는지 설명해 주세요.  
    - 사용자 질문에 대해, 구성원의 이력이 어떤 관점에서 유용하거나 참고할 만한지를 **구체적으로 설명**해 주세요.  
    - 어떤 프로젝트의 어떤 요소가 판단에 핵심적인 역할을 했는지 **명확히 근거를 제시**해 주세요.  
    - 마지막에, 질문자의 경력과 비교했을 때 조직 구성원의 이력에서 배울 수 있는 점이나 적용 가능한 인사이트를 종합적으로 제시하세요.

    [사용자 질문 또는 요청]
    {prompt}
    """.strip()


    response = client.chat.completions.create(
        model="gpt-4",  # 또는 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "당신은 프로젝트 기반 커리어 분석 전문가입니다."},
            {"role": "user", "content": full_prompt.strip()}
        ],
        temperature=0.5,
        max_tokens=1500
    )

    return response.choices[0].message.content.strip()

In [88]:
llm_answer = generate_llm_response_from_projects(columns, rows, prompt)
print("\n🧠 LLM의 분석 결과:")
print(llm_answer)


🧠 LLM의 분석 결과:
[1] 조직 구성원 경력 분석:

조직 구성원들은 다양한 프로젝트에서 다양한 역할을 수행하였습니다. 특히, EMP-010028은 IT 인프라, 온라인 서비스, 금융 등의 프로젝트에서 R2R, 개발, MPA 등의 역할을 수행하였습니다. 주요 기술로는 Infra PM, Financial Mgmt., PMO, ESG/SHE, SCM 등이 있습니다. EMP-010037은 통신사 G2 구축, 제2금융 Vitality 시스템 구축 및 운영, 제2금융 IDC 이전 및 통계인프라 시스템 구축 등의 프로젝트에서 AA, 팀장, 수행PM, 공통PL, 컨설팅 등의 역할을 수행하였습니다. 주요 기술로는 Mobile Dev., 자동화 Eng., Financial Mgmt., AIX, ESG/SHE 등이 있습니다. EMP-010040은 유통/서비스 차세대 ERP시스템 신규 구축 및 대리점 I/F 프로젝트, 정부기관 3.0 클라우드 사업 등의 프로젝트에서 AA, CA, 구축, 운영 등의 역할을 수행하였습니다. 주요 기술로는 Technical Architect, CRM, Infra PM, Factory 기획/설계, ERP_HCM 등이 있습니다.

[2] 질문자와 조직 구성원의 경력 비교:

질문자는 반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행하였으며, 주요 기술로는 자동화 Eng.를 보유하고 있습니다. 이와 비교해 볼 때, EMP-010037의 프로젝트 4에서 자동화 Eng. 기술을 사용하였고, EMP-010040의 프로젝트 7에서는 대규모, 장기 프로젝트를 경험하였다는 점에서 참고할 만한 경력을 가지고 있습니다.

[3] 질문자에게 유용한 구성원 이력:

EMP-010037의 프로젝트 4는 질문자의 경력과 유사한 자동화 Eng. 기술을 활용한 프로젝트입니다. 이 프로젝트에서는 IT 개발자 및 NMS에 대한 전반적인 구조, 환경 등에 대한 이해도를 향상시키는 커리어 임팩트를 얻었습니다. 이를 통해 질문자는 자신의 경